In [4]:
!pip install mysql-connector-python
!pip install matplotlib
!pip install requests

In [6]:
import mysql.connector
import matplotlib.pyplot as plt
import requests
from PIL import Image
import json
from io import BytesIO
import os

In [7]:
DB_HOST = ""
DB_USER = ""
DB_PASSWORD = ""
DB_NAME = ""

mydb = mysql.connector.connect(
  host=DB_HOST,
  user=DB_USER,
  password=DB_PASSWORD,
  database=DB_NAME,
  port=3306
)

False

In [ ]:
WIDTH = 480
HEIGHT = 360

In [ ]:
# get random visit id
query = ("SELECT visit_id FROM visit WHERE start_datetime >= %s AND start_datetime <= %s ORDER BY RAND() LIMIT 1")


start_datetime, end_datetime = "2024-10-03 00:00:00", "2024-10-31 23:59:59"

mycursor = mydb.cursor()
mycursor.execute(query, (start_datetime, end_datetime))
visit_id = mycursor.fetchone()[0]

VISIT_ID = visit_id

query = (
    "select photo_path, photo_row_number, photo_column_number, photo_path, json_ir from report_display_view_json rdvj where visit_id = %s"
)

query_2 = (
    "select photo_path, photo_row_number, photo_column_number, photo_path, json_ir from report_display_view_json_v3 rdvj where visit_id = %s"
)

mycursor = mydb.cursor()
mycursor.execute(query, (VISIT_ID,))
myresult = mycursor.fetchall()

mycursor = mydb.cursor()
mycursor.execute(query_2, (VISIT_ID,))
myresult_2 = mycursor.fetchall()

print("Visit ID: ", VISIT_ID)
print("Number of photos: ", len(myresult), " and ", len(myresult_2))

In [ ]:
for i in enumerate(myresult):
    photo_path_1, photo_row_number_1, photo_column_number_1, photo_path_1, json_ir_1 = myresult[i[0]]
    photo_path_2, photo_row_number_2, photo_column_number_2, photo_path_2, json_ir_2 = myresult_2[i[0]]

    data_1 = json.loads(json_ir_1)

    data_2 = json.loads(json_ir_2) if json_ir_2 else []


    # fig, ax = plt.subplots(figsize=(20, 12 ))
    fig, ax = plt.subplots(figsize=(8, 5))
    for item in data_1:
        box = item['boxes']
        object_name = item['object_name']

        y1, x1, y2, x2 = box




        x1 = x1 * WIDTH
        y1 = y1 * HEIGHT
        x2 = x2 * WIDTH
        y2 = y2 * HEIGHT


        rect_height = y2 - y1
        rect_width = x2 - x1


        rect = plt.Rectangle((x1, y1), rect_width, rect_height, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        plt.text(x1, y1, f"{item['object_name']} [{round(item['scores'], 2)}]", color='white', fontsize=8, backgroundcolor='red')


    response = requests.get(photo_path_1)

    img = Image.open(BytesIO(response.content))
    img = img.resize((WIDTH, HEIGHT))
    plt.imshow(img)
    plt.show()



    fig, ax = plt.subplots(figsize=(8, 5))
    for item in data_2:
        box = item['boxes']
        object_name = item['object_name']

        y1, x1, y2, x2 = box



        x1 = x1 * WIDTH
        y1 = y1 * HEIGHT
        x2 = x2 * WIDTH
        y2 = y2 * HEIGHT


        rect_height = y2 - y1
        rect_width = x2 - x1


        # rect = plt.Rectangle((box[0], box[1]), box[2] - box[0], box[3] - box[1], linewidth=1, edgecolor='r', facecolor='none')
        rect = plt.Rectangle((x1, y1), rect_width, rect_height, linewidth=1, edgecolor='g', facecolor='none')
        ax.add_patch(rect)
        plt.text(x1, y1, f"{item['object_name']} [{round(item['scores'], 2)}]", color='white', fontsize=8, backgroundcolor='green', rotation=15)
        # plt.text(x1, y1, f"{item['object_name']} [{round(item['scores'], 2)}]", color='white', fontsize=8, backgroundcolor='green')
        color = "blue" if round(item['scores']) >= 25 else "red"
        # if round(item['scores'] < 0.15:
        plt.text((x1 + x2) / 2, (y1 + y2) / 2, f"{round(item['scores'], 2)}%", color='white', fontsize=8, backgroundcolor=color, ha='center', va='center')


        # box[0] = 0 * HEIGHT
        # box[1] = 0 * WIDTH
        # box[2] = box[2] * HEIGHT
        # box[3] = box[3] * WIDTH

        # rect = plt.Rectangle((box[0], box[1]), box[2] - box[0], box[3] - box[1], linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        # plt.text(box[0], box[1], item['object_name'], color='white', fontsize=8, backgroundcolor='red')

    response = requests.get(photo_path_2)

    img = Image.open(BytesIO(response.content))
    img = img.resize((WIDTH, HEIGHT))
    plt.imshow(img)
    plt.show()